<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<h2>Deep Learning en el diagnóstico de Alzheimer mediante Imágenes de Resonancia Magnética</h2> 


## Instalar librerías

In [5]:
import warnings
warnings.filterwarnings('ignore')

## Importar librerías

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.exposure as e

from scipy.signal import convolve2d

In [ ]:
# TODO: PONER FUNCION GENERICA PARA IMPRIMIR 2 GRAFICAS DE COMPARACION

In [14]:
"""Función para normalizar imagen en escala de grises, traduce intensidad a 0-255 (escala de grises)

Parameters
----------
volume (image): numpy.ndarray 
cont: int

Returns: 
-------
img_resized: numpy.ndarray
"""

def normalize_image(volume:np.ndarray, count:int) -> np.ndarray:
    v_max = np.max(volume)
    v_min = np.min(volume)
    volume_norm = (volume - v_min) / (v_max - v_min)
    volume_norm = (volume_norm * 255).astype("int")
    
    if count == 0:
                    
        # Mostrarimagen normalizada
        fig = plt.figure(figsize=(10,10))

        # imágenes
        plt.subplot(2,2,1)
        plt.title('Imgen original')
        plt.axis('off')
        plt.imshow(volume, cmap='gray')

        plt.subplot(2,2,2)
        plt.title('Imagen normalizada')
        plt.axis('off')
        plt.imshow(volume_norm, cmap='gray')
    
    return volume_norm

# img = cv2.imread(r'D://TFM//Notebooks//Noise Reduction CVlib//Noise-Reduction1.jpg', 0)
# print(img.shape)
# img = normalize_image(img, 0)

In [16]:
"""Función para estandarizar tamaño de imagen 256 x 256.

Parameters
----------
img : numpy.ndarray
cont: int

Returns: 
-------
img_resized: numpy.ndarray
"""

def resize_img(img:np.ndarray, count:int) -> np.ndarray:
    
    width = 256
    height = 256
    dim = (width, height)
               
    img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
          
    if count == 0:

        # Mostrarimagen a la cual se le ha quitado el ruido (imagen mejorada)
        fig = plt.figure(figsize=(10,10))

        # imágenes
        plt.subplot(2,2,1)
        plt.title('Imgen tamaño original \n tamaño: ' + str(img.shape))
        plt.axis('off')
        plt.imshow(img, cmap='gray')

        plt.subplot(2,2,2)
        plt.title('Imagen tamaño estandar \n tamaño: ' + str(img_resized.shape))
        plt.axis('off')
        plt.imshow(img_resized, cmap='gray')
    
    return img_resized    
    
# img = resize_img(img, 0)
# print(img.shape)

In [17]:
"""Función para quitar el ruido a una imagen y suavizar.

Parameters
----------
img : numpy.ndarray
cont: int

Returns: 
-------
denoise_image: numpy.ndarray
"""

def reduce_noise(img:np.ndarray, count:int) -> np.ndarray:
    
    denoise_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(denoise_image.shape)
    denoise_image = cv2.fastNlMeansDenoisingColored(denoise_image, None, 5, 10, 7, 21)
    print(denoise_image.shape)
    denoise_image = cv2.cvtColor(denoise_image, cv2.COLOR_BGR2GRAY)
    print(denoise_image.shape)

    if count == 0:
        
        # Mostrarimagen a la cual se le ha quitado el ruido (imagen mejorada)
        fig = plt.figure(figsize=(10,10))

        # imágenes
        plt.subplot(2,2,1)
        plt.title('Imgen original')
        plt.axis('off')
        plt.imshow(img, cmap='gray')

        plt.subplot(2,2,2)
        plt.title('Imgen suavizada y sin ruido')
        plt.axis('off')
        plt.imshow(denoise_image, cmap='gray') 

    return denoise_image

# img = reduce_noise(img, 0)

In [19]:
"""Función para ecualizar el contraste de la imagen, mediante la distribución 
   de pixeles en el eje y y x de plano cartresiano o matriz de pixeles

Parameters
----------
img : numpy.ndarray
cont: int

Returns: 
-------
img_contraste_eq: numpy.ndarray
"""

def equalize_contrast(img:np.ndarray, count:int) -> np.ndarray:
    img_contraste_eq = e.equalize_adapthist(img)    

    if count == 0:
        
        fig = plt.figure(figsize=(10,10))
        
        # imágenes
        plt.subplot(2,2,1)
        plt.title('Imgen original')
        plt.axis('off')
        plt.imshow(img, cmap='gray')

        plt.subplot(2,2,2)
        plt.title('Imgen contrastada')
        plt.axis('off')
        plt.imshow(img_contraste_eq, cmap='gray') 
              
        # Histograma        
        H1, x1 = e.exposure.histogram(img)
        H2, x2 = e.exposure.histogram(img_contraste_eq)
       
        plt.subplot(2,2,3)
        plt.bar(x1, H1, width=0.4)
        plt.title('Distribucion pixeles imagen original')

        plt.subplot(2,2,4)
        plt.bar(x2, H2, width=0.4)
        plt.title('Distribucion pixeles imagen contrastada')
        
        plt.show()    
       
    return img_contraste_eq

# img = equalize_contrast(img, 0)
# print(img.shape)

In [7]:
"""Función imprimir diferencias entre una imagen original e imagen filtrada
   por medio de un filtro definido y convoluciones.

Parameters
----------
img : numpy.ndarray
cont: int
"""

def show_differences(kernel, image):
    convolved = convolve2d(image, kernel)
    fig = plt.figure(figsize=(10,10))
    plt.subplot(121)
    plt.title('Original image')
    plt.axis('off')
    plt.imshow(image, cmap='gray')
    
    plt.subplot(122)
    plt.title('Convolved image')
    plt.axis('off')
    plt.imshow(convolved, cmap='gray')
    return convolved  

In [8]:
"""Filtro para detectar o resaltar líneas verticales en imagen

Parameters
----------
img : numpy.ndarray
cont: int
"""

def detect_vertical_lines(img:np.ndarray, count:int) -> np.ndarray:
    
    if count == 0:

        print('Original Dimensions : ',img.shape)

        kernel_vertical = np.array([[-1, 0, 1],
                                    [-2, 0, 2],
                                    [-1, 0, 1]], np.float32)

        dx = show_differences(kernel_vertical, img)
    
# detect_vertical_lines(img,0)    

In [9]:
"""Filtro para detectar o resaltar líneas horizontales en imagen

Parameters
----------
img : numpy.ndarray
cont: int
"""

def detect_horizontal_lines(img:np.ndarray, count:int) -> np.ndarray:

    if count == 0:
    
        print('Original Dimensions : ',img.shape)

        kernel_horizontal = np.array([[1, 2, 1],
                                      [0, 0, 0],
                                      [-1, -2, -1]], np.float32)

        dx = show_differences(kernel_horizontal, img)
    
# detect_horizontal_lines(img,0)  